In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from random import randint, uniform

In [2]:
# === Config ===
OUTPUT_DIR = "synthetic_motifs"
IMG_SIZE = 512
IMAGES_PER_CLASS = 10
motif_classes = [
    "zigzag", "chevron", "spiral", "crosshatch", "dotted",
    "horizontal_lines", "vertical_lines", "meander", "wave", "comb"
]

In [3]:
os.makedirs(os.path.join(OUTPUT_DIR, "images"), exist_ok=True)

metadata = []

In [4]:
def draw_zigzag(draw):
    spacing = randint(40, 80)
    for y in range(0, IMG_SIZE, spacing):
        points = [(x, y + (x % (2 * spacing))) for x in range(0, IMG_SIZE, spacing)]
        draw.line(points, fill=255, width=randint(5, 15))

def draw_chevron(draw):
    spacing = randint(40, 80)
    for y in range(0, IMG_SIZE, spacing):
        for x in range(0, IMG_SIZE, spacing*2):
            draw.line([(x, y), (x + spacing, y + spacing), (x + 2*spacing, y)], fill=255, width=randint(5, 15))

def draw_spiral(draw):
    center = (IMG_SIZE // 2, IMG_SIZE // 2)
    r = 10
    angle = 0
    points = []
    while r < IMG_SIZE // 2:
        x = int(center[0] + r * np.cos(angle))
        y = int(center[1] + r * np.sin(angle))
        points.append((x, y))
        r += 0.5
        angle += 0.1
    draw.line(points, fill=255, width=8)

def draw_crosshatch(draw):
    spacing = randint(30, 70)
    for i in range(0, IMG_SIZE, spacing):
        draw.line([(i, 0), (0, i)], fill=255, width=randint(5, 10))
        draw.line([(IMG_SIZE, i), (i, IMG_SIZE)], fill=255, width=randint(5, 10))

def draw_dotted(draw):
    spacing = randint(40, 80)
    for y in range(0, IMG_SIZE, spacing):
        for x in range(0, IMG_SIZE, spacing):
            radius = randint(5, 15)
            draw.ellipse([x-radius, y-radius, x+radius, y+radius], fill=255)

def draw_horizontal_lines(draw):
    spacing = randint(30, 80)
    for y in range(0, IMG_SIZE, spacing):
        draw.line([(0, y), (IMG_SIZE, y)], fill=255, width=randint(5, 10))

def draw_vertical_lines(draw):
    spacing = randint(30, 80)
    for x in range(0, IMG_SIZE, spacing):
        draw.line([(x, 0), (x, IMG_SIZE)], fill=255, width=randint(5, 10))

def draw_meander(draw):
    spacing = randint(50, 100)
    for y in range(0, IMG_SIZE, spacing * 2):
        for x in range(0, IMG_SIZE, spacing * 2):
            draw.rectangle([x, y, x + spacing, y + spacing], outline=255, width=randint(5, 10))

def draw_wave(draw):
    amplitude = randint(20, 50)
    frequency = uniform(0.01, 0.03)
    for y_offset in range(0, IMG_SIZE, 100):
        points = [(x, int(IMG_SIZE//2 + amplitude * np.sin(frequency * x + y_offset))) for x in range(0, IMG_SIZE)]
        draw.line(points, fill=255, width=randint(5, 10))

def draw_comb(draw):
    base_y = IMG_SIZE // 2
    spacing = randint(40, 80)
    draw.line([(0, base_y), (IMG_SIZE, base_y)], fill=255, width=randint(5, 10))
    for x in range(0, IMG_SIZE, spacing):
        draw.line([(x, base_y), (x, base_y - spacing)], fill=255, width=randint(5, 10))

In [5]:
motif_functions = {
    "zigzag": draw_zigzag,
    "chevron": draw_chevron,
    "spiral": draw_spiral,
    "crosshatch": draw_crosshatch,
    "dotted": draw_dotted,
    "horizontal_lines": draw_horizontal_lines,
    "vertical_lines": draw_vertical_lines,
    "meander": draw_meander,
    "wave": draw_wave,
    "comb": draw_comb,
}

In [6]:
# === Generate Images ===
for motif in motif_classes:
    for i in range(IMAGES_PER_CLASS):
        img = Image.new("L", (IMG_SIZE, IMG_SIZE), 0)
        draw = ImageDraw.Draw(img)
        motif_functions[motif](draw)

        # Simulate aging/wear
        img_np = np.array(img)
        img_np = cv2.GaussianBlur(img_np, (5, 5), 0)
        _, img_np = cv2.threshold(img_np, 30, 255, cv2.THRESH_BINARY)
        img_out = Image.fromarray(img_np)

        fname = f"{motif}_{i:03d}.png"
        img_out.save(os.path.join(OUTPUT_DIR, "images", fname))
        metadata.append({
            "image_id": fname,
            "motif_style": motif
        })

In [7]:
# === Save Metadata ===
df = pd.DataFrame(metadata)
df.to_csv(os.path.join(OUTPUT_DIR, "metadata.csv"), index=False)

print("✅ Synthetic dataset generated.")


✅ Synthetic dataset generated.
